![Cabec%CC%A7alho_notebook.png](cabecalho_notebook.png)

# PCA - Tarefa 01: *HAR* com PCA

Vamos trabalhar com a base da demonstração feita em aula, mas vamos explorar um pouco melhor como é o desempenho da árvore variando o número de componentes principais.

In [6]:
import pandas as pd

from sklearn.tree import DecisionTreeClassifier

from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

filename_features = "../Dados/UCI HAR Dataset/UCI HAR Dataset/features.txt"
filename_labels = "../Dados/UCI HAR Dataset/UCI HAR Dataset/activity_labels.txt"

filename_subtrain = "../Dados/UCI HAR Dataset/UCI HAR Dataset/train/subject_train.txt"
filename_xtrain = "../Dados/UCI HAR Dataset/UCI HAR Dataset/train/X_train.txt"
filename_ytrain = "../Dados/UCI HAR Dataset/UCI HAR Dataset/train/y_train.txt"

filename_subtest = "../Dados/UCI HAR Dataset/UCI HAR Dataset/test/subject_test.txt"
ffilename_xtest = "../Dados/UCI HAR Dataset/UCI HAR Dataset/test/X_test.txt"
filename_ytest = "../Dados/UCI HAR Dataset/UCI HAR Dataset/test/y_test.txt"

features = pd.read_csv(filename_features, header=None, names=['nome_var'], sep="#")
labels = pd.read_csv(filename_labels, sep=r'\s+', header=None, names=['cod_label', 'label'])

subject_train = pd.read_csv(filename_subtrain, header=None, names=['subject_id'])
X_train = pd.read_csv(filename_xtrain, sep=r'\s+', header=None, names=features['nome_var'].tolist())
y_train = pd.read_csv(filename_ytrain, header=None, names=['cod_label'])

subject_test = pd.read_csv(filename_subtest, header=None, names=['subject_id'])
X_test = pd.read_csv(ffilename_xtest, sep=r'\s+', header=None, names=features['nome_var'].tolist())
y_test = pd.read_csv(filename_ytest, header=None, names=['cod_label'])

## Árvore de decisão

Rode uma árvore de decisão com todas as variáveis, utilizando o ```ccp_alpha=0.001```. Avalie a acurácia nas bases de treinamento e teste. Avalie o tempo de processamento.

In [8]:
%%time

dt = DecisionTreeClassifier(ccp_alpha=0.001, random_state=42)
dt.fit(X_train, y_train['cod_label'])

y_train_pred = dt.predict(X_train)
train_accuracy = accuracy_score(y_train['cod_label'], y_train_pred)

y_test_pred = dt.predict(X_test)
test_accuracy = accuracy_score(y_test['cod_label'], y_test_pred)

print(f"Acurácia na base de treinamento: {train_accuracy:.4f}")
print(f"Acurácia na base de teste: {test_accuracy:.4f}")


Acurácia na base de treinamento: 0.9758
Acurácia na base de teste: 0.8799
CPU times: total: 6.03 s
Wall time: 7.26 s


## Árvore com PCA

Faça uma análise de componemtes principais das variáveis originais. Utilize apenas uma componente. Faça uma árvore de decisão com esta componente como variável explicativa.

- Avalie a acurácia nas bases de treinamento e teste
- Avalie o tempo de processamento

In [10]:
%%time
pca = PCA(n_components=1)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

dt_pca = DecisionTreeClassifier(ccp_alpha=0.001, random_state=42)
dt_pca.fit(X_train_pca, y_train['cod_label'])

y_train_pred_pca = dt_pca.predict(X_train_pca)
train_accuracy_pca = accuracy_score(y_train['cod_label'], y_train_pred_pca)

y_test_pred_pca = dt_pca.predict(X_test_pca)
test_accuracy_pca = accuracy_score(y_test['cod_label'], y_test_pred_pca)

print(f"Acurácia na base de treinamento: {train_accuracy_pca:.4f}")
print(f"Acurácia na base de teste: {test_accuracy_pca:.4f}")

Acurácia na base de treinamento: 0.4997
Acurácia na base de teste: 0.4571
CPU times: total: 391 ms
Wall time: 263 ms


## Testando o número de componentes

Com base no código acima, teste a árvore de classificação com pelo menos as seguintes possibilidades de quantidades de componentes: ```[1, 2, 5, 10, 50]```. Avalie para cada uma delas:

- Acurácia nas bases de treino e teste
- Tempo de processamento


In [13]:
%%time

n = [1, 2, 5, 10, 50]
train_accuracies = []
test_accuracies = []

for n_comp in n:

    pca = PCA(n_components=n_comp)
    X_train_pca = pca.fit_transform(X_train)
    X_test_pca = pca.transform(X_test)
    
    dt_model = DecisionTreeClassifier(ccp_alpha=0.001, random_state=42)
    dt_model.fit(X_train_pca, y_train['cod_label'])
    
    y_train_pred = dt_model.predict(X_train_pca)
    train_acc = accuracy_score(y_train['cod_label'], y_train_pred)
    train_accuracies.append(train_acc)
    
    # Avaliando na base de teste
    y_test_pred = dt_model.predict(X_test_pca)
    test_acc = accuracy_score(y_test['cod_label'], y_test_pred)
    test_accuracies.append(test_acc)
    
    # Imprimindo resultados
    print(f"\nNumero de componentes: {n_comp}")
    print(f"Variância explicada acumulada: {var_exp:.4f} ({var_exp*100:.2f}%)")
    print(f"Acurácia no treino: {train_acc:.4f}")
    print(f"Acurácia no teste: {test_acc:.4f}")


Numero de componentes: 1
Variância explicada acumulada: 0.6255 (62.55%)
Acurácia no treino: 0.4997
Acurácia no teste: 0.4571

Numero de componentes: 2
Variância explicada acumulada: 0.6255 (62.55%)
Acurácia no treino: 0.6128
Acurácia no teste: 0.5847

Numero de componentes: 5
Variância explicada acumulada: 0.6255 (62.55%)
Acurácia no treino: 0.8460
Acurácia no teste: 0.7886

Numero de componentes: 10
Variância explicada acumulada: 0.6255 (62.55%)
Acurácia no treino: 0.8935
Acurácia no teste: 0.8215

Numero de componentes: 50
Variância explicada acumulada: 0.6255 (62.55%)
Acurácia no treino: 0.9162
Acurácia no teste: 0.8249
CPU times: total: 4.16 s
Wall time: 1.99 s


## Conclua

- O que aconteceu com a acurácia?
- O que aconteceu com o tempo de processamento?

- A acurácia aumentou consistentemente à medida que o número de componentes principais aumentou, tanto na base de treino quanto na de teste:

    - Com 1 componente: 49.97% (treino) e 45.71% (teste)
    - Com 50 componentes: 91.62% (treino) e 82.49% (teste)

- Os maiores aumentos na acurácia ocorreram nas primeiras adições de componentes:

    - De 1 para 2 componentes: aumento de ~11% no treino e ~13% no teste
    - De 2 para 5 componentes: aumento de ~23% no treino e ~20% no teste
    - De 5 para 10 componentes: aumento de ~4.8% no treino e ~3.3% no teste
    - De 10 para 50 componentes: aumento de apenas ~2.3% no treino e ~0.3% no teste

- O ganho em acurácia diminui significativamente à medida que mais componentes são adicionados, especialmente após 10 componentes. A diferença entre acurácia de treino e teste aumentou com mais componentes, indicando um possível início de sobreajuste à medida que o modelo se torna mais complexo.


Olhando para o tempo de processamento, mesmo levando em conta que foi treinado varios modelos, ele aumentou significamente, de 300 ms para 4.16s. Enquanto que o tempo de processamento sem a utilização de pca foi de 6 segundos, mostrando uma redução mantendo a acuracia em valores satisfatórios.